In [ ]:
# things that dont change - source for copy-pasting to update new consume cell each iteration...
# this is a spin-off of mikestestingnotebook.ipynb...   tailored for the Managed Endpoint Deployments using the CWD ONNX model
# compute
# compute-ds11v2-aivision

# kernel
# Python 3.10 - SDK v2


data = list({"./TestImage.jpg"}) # <-- THIS creates a LIST from the set, and that list IS json serializable....  should be a correction in run time no rebuild...

# data = list({"https://asatestimages01.blob.core.windows.net/" \
# "container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg"}) 
# # THIS creates a LIST from the set, and that list IS json serializable....  should be a correction in run time no rebuild...
# print("data = ", data) # this helps debug by printing data so we can see full URL...


# MIKE MAKING ERROR HERE...   data should be a key: value pair...   
image_url = "https://asatestimages01.blob.core.windows.net/" \
"container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 

urlData = {
    "image_url": image_url
}

# body = str.encode(json.dumps(data))
body = str.encode(json.dumps(urlData))





In [10]:
# 2023Nov20 mcvogt
# this has been tested and is fully functional to send a test image stored in Azure Storage Account, to and Azure Managed Endpoint hosting an exported 
# Azure Custom Vision model

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# LIST of co-located test images, available for one-off and small-batch testing... 
image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 

urlData = {
    "image_url": image_url
}
body = str.encode(json.dumps(urlData))

url = 'https://aivisionexperiment01-url021c2.northcentralus.inference.ml.azure.com/score'

api_key = 'g7jfXX9VjePlk3eaenh1whtGgysX31Z1'

if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mikecwdodonnxmodelfromotherfr-1' , 'detThresh': '0.10'}
# headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mikecwdodonnxmodelfromotherfr-1' }
# MIKE - this calling structure appears to be friendly to additiona key: value pairs injected into 'heders' and pulled out on the score.py recieving end
# to dynamically set the display Proabability Threshold w each call of score.py's run().  work that shit out if asked...
# clearly mike can introduce this IF he uses ascii text ('0.10') to send a numerical value (0.10) (convert string to float over in receiving score.py )

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

# ---------------------------------- end of code --------------------------------------





b'"{\\"detected_boxes\\": [0.026675760746002197, 0.04183569550514221, 0.8069154620170593, 0.814767599105835], \\"detected_classes\\": [0], \\"detected_scores\\": [0.018364280462265015]}"'


# MIKE - it would be a good idea to pass in Probability Threshold in as a header key:value pair... so at run time the app developer can alter this...  


In [6]:
# this is mikes cell for processing multiple images... 

# 2023Nov20 mcvogt
# this has been tested and is fully functional to send a test image stored in Azure Storage Account, to and Azure Managed Endpoint hosting an exported 
# Azure Custom Vision model

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# LIST of co-located test images, available for one-off and small-batch testing... 
# image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 


image_host = "https://asatestimages01.blob.core.windows.net/"
image_container = "container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/"

image_file_set = {
"UnHealthySSWI000000012444874A.jpg", 
"UnHealthySSWI000000012733278A.jpg", 
"UnHealthySSWI000000012931348B.jpg", 
"UnHealthySSWI000000013042672A.jpg", 
"UnHealthySSWI000000013486750A.jpg",
"HealthySSWI000000016989655A.jpg", 
"HealthySSWI000000017061751A.jpg", 
"HealthySSWI000000017167085C.jpg", 
"HealthySSWI000000016685494B.jpg", 
"HealthySSWI000000016182548A.jpg"
}

print("\nFull Elements of image_file_set: ")
for i in image_file_set:
        image_url = (image_host + image_container + i)
        print("\n")
        print(image_url)
        # NOTE - all rest of constructing POST request remains identical except indented 1 more level to be subordinate to 'for i' loop... 
        urlData = {
            "image_url": image_url
        }
        body = str.encode(json.dumps(urlData))

        url = 'https://aivisionexperiment01-url021c2.northcentralus.inference.ml.azure.com/score'

        api_key = 'g7jfXX9VjePlk3eaenh1whtGgysX31Z1'

        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mikecwdodonnxmodelfromotherfr-1' }

        req = urllib.request.Request(url, body, headers)

        try:
            response = urllib.request.urlopen(req)
            result = response.read()
            print(result)
        except urllib.error.HTTPError as error:
            print("The request failed with status code: " + str(error.code))
            print(error.info())
            print(error.read().decode("utf8", 'ignore'))

# ---------------------------------- end of code --------------------------------------






Full Elements of image_file_set: 


https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/UnHealthySSWI000000012444874A.jpg
b'"{\\"detected_boxes\\": [0.2706890404224396, 0.5266299247741699, 0.6669864654541016, 0.8851373195648193, 0.0010432451963424683, 0.5401623249053955, 0.30675649642944336, 0.9181697368621826, 0.08526048064231873, 0.49914222955703735, 0.608391284942627, 0.9290570616722107], \\"detected_classes\\": [1, 1, 0], \\"detected_scores\\": [0.999832034111023, 0.10428383946418762, 0.012935787439346313]}"'


https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Mixed_Deer_Images_for_Testing/HealthySSWI000000017167085C.jpg
b'"{\\"detected_boxes\\": [0.50816410779953, 0.2906777858734131, 0.9812902808189392, 0.9853470325469971, 0.17043524980545044, 0.22910627722740173, 0.8411441445350647, 1.0016082525253296], \\"detected_classes\\": [1, 0], \\"detected_scores\\": [0.9520444869995117, 0.0488

In [16]:
# this is code cell for 021c4 !!!!!!!!!!
#021c4 was 021c, reDeployed for Curated ENv pytorch2.0 but only 1 X E4s_v3 SKU.. so should be lowest cost to operate...

import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

image_url = "https://asatestimages01.blob.core.windows.net/" \
"container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 

urlData = {
    "image_url": image_url
}

body = str.encode(json.dumps(urlData))

url = 'https://aivisionexperiment01-url021c4.northcentralus.inference.ml.azure.com/score'
api_key = 'PXR12Tx4kt5wU7fdIbG933GMkNcwdMed'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mikecwdodonnxmodelfromotherfr-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"detected_boxes\\": [0.026675760746002197, 0.04183569550514221, 0.8069154620170593, 0.814767599105835], \\"detected_classes\\": [0], \\"detected_scores\\": [0.018364280462265015]}"'


In [22]:
# 2023Nov20 mcvogt
# this is code cell for 021c3     uses CurEnv pytorch1.13 + 1 X E4s_v3 SKU
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

image_url = "https://asatestimages01.blob.core.windows.net/container-testimages/DEER_CWD_imagery/Cropped_IC_Model/DeerDayHealthyCandidates/TestImageBlob.jpg" 

urlData = {
    "image_url": image_url
}

body = str.encode(json.dumps(urlData))

url = 'https://aivisionexperiment01-url021c3.northcentralus.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '8GsGGBVpod98cjxxXHFeMKyzvElZ4Eaz'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'mikecwdodonnxmodelfromotherfr-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"detected_boxes\\": [0.026675760746002197, 0.04183569550514221, 0.8069154620170593, 0.814767599105835], \\"detected_classes\\": [0], \\"detected_scores\\": [0.018364280462265015]}"'
